In [1]:
import numpy as np
from scipy.integrate import solve_ivp

import plotly.graph_objects as go

# parameters
sigma = 10
b = 8/3
r = 28

Ts = [10, 100, 1000]
x0 = [1, 1, 1]

epsilons = [0, 1e-6]

# Equations
lorenz_equations = lambda t, x: [
    -sigma * x[0] + sigma * x[1],
    - x[0]*x[2] + r * x[0] - x[1],
    x[0]*x[1] - b*x[2]
]

In [2]:
# Solve
solutions = dict()
for T in Ts:
    for epsilon in epsilons:
        sol = solve_ivp(lorenz_equations, [0, T], [x + epsilon for x in x0], dense_output=True)
        solutions[(T, epsilon)] = sol

In [18]:
def plot(solution, n_points, caption):
    # Plots a solution
    t_max = solution.sol.t_max
    t_min = solution.sol.t_min
    ts = np.linspace(t_min, t_max, n_points)
    sol = solution.sol(ts)

    fig = go.Figure(data=go.Scatter3d(
        x=sol[0], y=sol[1], z=sol[2],
        marker=dict(
            size=1,
            color=ts,
            colorscale='Viridis',
        ),
        line=dict(
            color=ts,
            colorscale='Viridis',
            width=2
        )
    ))

    fig.update_layout(
        title='Lorenz System ' + caption,
        title_x=0.5,
        width=700,
        height=700,
        autosize=False,
        scene=dict(
            camera=dict(
                up=dict(
                    x=0,
                    y=0,
                    z=1
                ),
                eye=dict(
                    x=1.4,
                    y=-1.4,
                    z=1.4,
                )
            ),
            aspectratio = dict( x=1, y=1, z=1 ),
            aspectmode = 'manual',
        ),
    )
    return fig

In [19]:
key = (100,0)
fig = plot(solutions[key], n_points=1e3, caption="T=%d, ε=%d"%key)
#fig.write_image("images/lorenz-%s-%s.pdf"%key)
fig.show()

In [20]:
for key, value in solutions.items():
    fig = plot(value, n_points=1e5, caption="T=%d, ε=%.2E"%key)
    fig.write_image("images/lorenz-%s-%.2E.pdf"%key)
    #fig.show()